In [7]:

import torch
import numpy as np
import scipy as sp
import sys
sys.path.append('/Users/judd/Documents/optimisation_of_superconduncting_circuits/core')
from fluxonium import Fluxonium 
import general as general


In [8]:
#GLOBAL PARAMETERS
R_k = sp.constants.h / (sp.constants.e**2.0)
omega_low = 1e-9 * 2 * np.pi  # Low frequency cutoff. Units: 2pi GHz
omega_high = 3 * 2 * np.pi  # High frequency cutoff. Units: 2pi GHz
t_exp = 1e4  #Measurement time. Units: ns
A_cc = 1e-7 # Critical current noise strength. Units of critical current I_c
A_flux = 1e-6   # Flux noise strength. Units: Phi_0
M= 400  # Mutual inductance between qubit and a flux line. Units: \Phi_0 / Ampere
R_0 = 50 # Characteristic impedance of a transmission line. Units: ohms
T = 0.015 # Typical temperature for a superconducting circuit experiment. Units: K



In [10]:
EJ = 10
EC = 2.5
EL = 1.4
flux = 0.5
dim = 55


fluxonium = Fluxonium(EJ, EC, EL, flux, dim, 'auto_H')
fluxonium2 = Fluxonium(EJ, EC, EL, flux, dim, 'create_H')


def effective_t1_time(fluxonium):

    eigvals,eigvecs = fluxonium.esys()

    t1_rate = torch.zeros([1,1],dtype=torch.double)

    #t1_capacitive

    t1_rate += general.t1(noise_op = fluxonium.n_op(), spectral_density = fluxonium.spectral_density_cap(T), eigvecs = eigvecs)

    #charge impendenace not included?

    #t1_flux_bias_line

    t1_rate += general.t1(noise_op = fluxonium.d_ham_d_flux(), spectral_density = fluxonium.spectral_density_fbl(T, M, R_0), eigvecs = eigvecs)

    #3.4 t1_inductive

    t1_rate += general.t1(noise_op = fluxonium.phi()+0j, spectral_density = fluxonium.spectral_density_ind(T), eigvecs = eigvecs)

    #3.5 t1_quasiparticle_tunneling - a little dubious

    t1_rate += general.t1(noise_op = fluxonium.qt_noise_op()+0j, spectral_density = fluxonium.spectral_density_qt(T, R_k), eigvecs = eigvecs)

    return t1_rate


def effective_tphi_time(fluxonium):

    eigvals,eigvecs = fluxonium.esys()

    tphi_rate = torch.zeros([1,1],dtype=torch.double)

    #tphi_1_over_f_flux

    tphi_rate += general.tphi(A_flux, fluxonium.d_ham_d_flux(),  eigvecs=eigvecs)
    
    #tphi_1_over_f_cc

    tphi_rate += general.tphi(A_cc, fluxonium.d_ham_d_EJ(), eigvecs=eigvecs)

    return  tphi_rate


def t2():
    
    return 1/(0.5*effective_t1_time(fluxonium))+1/effective_tphi_time(fluxonium)


t2





tensor([[4952760.4323]], dtype=torch.float64)